# Training

Select training_env as the kernel (Kernel -> change kernel) . If it is not available, refer to the README.md section Training a model. You will need to install the anaconda environment :  training_env.
Change the path in the Notebook.
Moreover to check in realtime the evolution of the loss and metrics look at the README.md section detailling the process with Tensorboard. The saved logs will be at the location PATH_THIS_TRAINING+"logs"
Eventually to visualize the image generated look at the notebook Visualize_training_data.ipynb


In [2]:
import sys

In [3]:
sys.path.append("/home/idumeur/code")
sys.path.append("/home/idumeur/code/sent2_cloud_remover/")

In [4]:
from utils.load_dataset import load_data
from models import clean_gan
from train import open_yaml,saving_yaml

from tensorflow.python.client import device_lib

In [ ]:
PATH="/srv/osirim/idumeur/data/dataset6/prepro1/input_large_dataset/" #path to the dataset which contains the tile for the training
PATH_TRAININGS="/srv/osirim/idumeur/trainings/"
NAME_MODEL="new_model" #name of the model
TRAINING_NBER="9" #Id of the training
PATH_THIS_MODEL=PATH_TRAININGS+NAME_MODEL+"/"
PATH_THIS_TRAINING="{}{}/training_{}/".format(PATH_TRAININGS,NAME_MODEL,TRAINING_NBER)
PATH_CHECKPOINT="{}checkpoints/".format(PATH_THIS_TRAINING)
PATH_SAVED_IM="{}saved_training_images/".format(PATH_THIS_TRAINING)
PATH_TRAIN_YAML="/home/idumeur/code/sent2_cloud_remover/GAN_confs/train.yaml" #The based configuratiion file for the training param
PATH_MODEL_YAML="/home/idumeur/code/sent2_cloud_remover/GAN_confs/model.yaml" #The based configuratiion file for the model param

In [6]:
train_param=open_yaml(PATH_TRAIN_YAML) #dict
model_param=open_yaml(PATH_MODEL_YAML)

/home/idumeur/code/sent2_cloud_remover/train.py:43: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  return yaml.load(f)


In [7]:
model_param["model_name"]=NAME_MODEL

## Discriminator
here are the current parameters for the discriminator convolutional layers. You can change them by setting by model_param["dict_discri_archi"]= new_dictionnary. In the dictionnary, keys are the number of the conv layer, and the list corresponds respectively to ["padding","stride","kernel","nfilter"]

In [8]:
model_param["dict_discri_archi"]

{1: [1, 2, 4, 64],
 2: [1, 2, 4, 256],
 3: [1, 2, 4, 156],
 4: [1, 1, 4, 512],
 5: [1, 1, 4, 1]}

In [9]:
model_param["d_last_activ"] #The last activation of the discriminator
model_param["real_label_smoothing"]=[1.0,1.0]
model_param["fake_label_smoothing"]=[0.0,0.0]


In [10]:
#Set Training parameters
train_param["train_directory"]=PATH+"train/" #path to the train data
train_param["val_directory"]=PATH+"val/" #path to the val data
train_param["training_number"]=TRAINING_NBER #id of the training
train_param["epoch"]=500 #max nber of epochs
train_param["lambda"]=0 #factor for the L1 loss
train_param["lr"]=0.001

In [11]:
train_param["normalization"]=True #id set to true normalization to the data is apllied, the norm implemented is standardization for SAR data and normalization for RGBNIR
#train_param["dict_band_x"]=None #the stats for normalization procedure is going to be compute on each of these group of bands
#train_param["dict_band_label"]=None
#train_param["dict_rescale_type"]=None
train_param["training_dir"]=PATH_TRAININGS
train_param["s2_scale"]=1/7 #for the normalization, do not change
train_param["s1_scale"]=1/5 #for the normalization, do not change
train_param["s2bands"]=["R,G,B","NIR"] #for the normalization, do not change

In [13]:
train_param["lim_train_tile"]=10
train_param["lim_val_tile"]=3

In [14]:
# Option for tracking the training 
train_param["im_saving_step"]=10 #every n epochs image from the training set are saved
train_param["weights_saving_step"]=50 # every n epoch, the model is saved
train_param["metric_step"]=10 #every n epochs, the metrics will be computed and displayed with tensorboard

In [15]:
#load data and model 
gan=clean_gan.GAN(model_param, train_param)

Remove the negative values in order to have no error in the log : negative value will be replaced usingknn algorithm
Important the index of the bands in lband_index should be index that follow each other
No scaler was defined before
Remove the negative values in order to have no error in the log : negative value will be replaced usingknn algorithm
Important the index of the bands in lband_index should be index that follow each other
No scaler was defined before
No scaler was defined before
No scaler was defined before
Remove the negative values in order to have no error in the log : negative value will be replaced usingknn algorithm
Important the index of the bands in lband_index should be index that follow each other
Remove the negative values in order to have no error in the log : negative value will be replaced usingknn algorithm
Important the index of the bands in lband_index should be index that follow each other
Loading the data done dataX (10, 256, 256, 8) dataY (10, 256, 256, 4

G Tensor("Generator/Identity:0", shape=(None, 256, 256, 4), dtype=float32)
INPUT DISCRI  Tensor("concat:0", shape=(None, 256, 256, 12), dtype=float32)
[INFO] combined model loss are : 


In [16]:
#check available ressources

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6987912960863644197
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7632721629351614333
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11211344448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15102665228234073574
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:03:00.0, compute capability: 3.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8528616695278607355
physical_device_desc: "device: XLA_GPU device"
]


In [ ]:
#Start the training
model_dir = gan.model_dir
training_dir = gan.this_training_dir
#saving_yaml(path_model, model_dir)
#saving_yaml(path_train, training_dir)
gan.train()

0 iter 0 [D loss: 0.798994, acc.: 49.00%] [G loss: 0.573215 0.573215]


/home/idumeur/code/sent2_cloud_remover/utils/metrics.py:56: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  lssim_batch += [ssim(batch1[i, :, :, :], batch2[i, :, :, :], multichannel=True)]
/home/idumeur/miniconda3/envs/training_env/lib/python3.6/site-packages/skimage/metrics/_structural_similarity.py:108: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  im2[..., ch], **args)


0 iter 1 [D loss: 0.806540, acc.: 47.78%] [G loss: 0.365085 0.365085]
0 iter 2 [D loss: 0.793997, acc.: 52.00%] [G loss: 0.291942 0.291942]
0 iter 3 [D loss: 0.789682, acc.: 47.22%] [G loss: 0.199970 0.199970]
0 iter 4 [D loss: 0.777531, acc.: 48.06%] [G loss: 0.553296 0.553296]
0 iter 5 [D loss: 0.784808, acc.: 50.06%] [G loss: 0.195326 0.195326]
0 iter 6 [D loss: 0.780038, acc.: 50.39%] [G loss: 0.167596 0.167596]
0 iter 7 [D loss: 0.769406, acc.: 52.50%] [G loss: 0.845658 0.845658]
0 iter 8 [D loss: 0.779096, acc.: 64.06%] [G loss: 0.011596 0.011596]
0 iter 9 [D loss: 0.772808, acc.: 44.94%] [G loss: 0.444333 0.444333]
Saving model at /srv/osirim/idumeur/trainings/new_model/training_9/checkpoints/ step 0
1 iter 10 [D loss: 0.787733, acc.: 49.33%] [G loss: 0.099657 0.099657]
1 iter 11 [D loss: 0.784751, acc.: 42.17%] [G loss: 0.054520 0.054520]
1 iter 12 [D loss: 0.780950, acc.: 52.00%] [G loss: 0.456693 0.456693]
1 iter 13 [D loss: 0.776485, acc.: 36.00%] [G loss: 0.541178 0.541178]

11 iter 115 [D loss: 0.717836, acc.: 53.50%] [G loss: 0.772792 0.772792]
11 iter 116 [D loss: 0.709521, acc.: 48.06%] [G loss: 0.737469 0.737469]
11 iter 117 [D loss: 0.711638, acc.: 51.56%] [G loss: 0.781883 0.781883]
11 iter 118 [D loss: 0.715560, acc.: 60.44%] [G loss: 0.760207 0.760207]
11 iter 119 [D loss: 0.716599, acc.: 46.83%] [G loss: 0.764892 0.764892]
12 iter 120 [D loss: 0.721346, acc.: 64.67%] [G loss: 0.763846 0.763846]
12 iter 121 [D loss: 0.715715, acc.: 45.00%] [G loss: 0.776274 0.776274]
12 iter 122 [D loss: 0.710528, acc.: 48.39%] [G loss: 0.810687 0.810687]
12 iter 123 [D loss: 0.712252, acc.: 46.00%] [G loss: 0.783228 0.783228]
12 iter 124 [D loss: 0.715224, acc.: 42.28%] [G loss: 0.767156 0.767156]
12 iter 125 [D loss: 0.719056, acc.: 55.11%] [G loss: 0.761735 0.761735]
12 iter 126 [D loss: 0.707670, acc.: 39.78%] [G loss: 0.708022 0.708022]
12 iter 127 [D loss: 0.711068, acc.: 50.89%] [G loss: 0.779043 0.779043]
12 iter 128 [D loss: 0.713869, acc.: 53.22%] [G los

22 iter 228 [D loss: 0.705792, acc.: 49.61%] [G loss: 0.698757 0.698757]
22 iter 229 [D loss: 0.705806, acc.: 52.94%] [G loss: 0.705424 0.705424]
23 iter 230 [D loss: 0.705838, acc.: 47.33%] [G loss: 0.718748 0.718748]
23 iter 231 [D loss: 0.705769, acc.: 53.72%] [G loss: 0.712706 0.712706]
23 iter 232 [D loss: 0.705651, acc.: 49.50%] [G loss: 0.707458 0.707458]
23 iter 233 [D loss: 0.705848, acc.: 50.11%] [G loss: 0.696636 0.696636]
23 iter 234 [D loss: 0.705816, acc.: 48.78%] [G loss: 0.682330 0.682330]
23 iter 235 [D loss: 0.705859, acc.: 51.44%] [G loss: 0.694247 0.694247]
23 iter 236 [D loss: 0.705623, acc.: 48.28%] [G loss: 0.718509 0.718509]
23 iter 237 [D loss: 0.705735, acc.: 51.83%] [G loss: 0.682881 0.682881]
23 iter 238 [D loss: 0.705747, acc.: 46.78%] [G loss: 0.693302 0.693302]
23 iter 239 [D loss: 0.705717, acc.: 53.50%] [G loss: 0.703698 0.703698]
24 iter 240 [D loss: 0.705723, acc.: 47.83%] [G loss: 0.708571 0.708571]
24 iter 241 [D loss: 0.705700, acc.: 50.94%] [G los

...

215 iter 2155 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691544 0.691544]
215 iter 2156 [D loss: 0.693232, acc.: 50.00%] [G loss: 0.691546 0.691546]
215 iter 2157 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691547 0.691547]
215 iter 2158 [D loss: 0.693232, acc.: 50.00%] [G loss: 0.691549 0.691549]
215 iter 2159 [D loss: 0.693232, acc.: 50.00%] [G loss: 0.691551 0.691551]
216 iter 2160 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691552 0.691552]
216 iter 2161 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691554 0.691554]
216 iter 2162 [D loss: 0.693232, acc.: 50.00%] [G loss: 0.691556 0.691556]
216 iter 2163 [D loss: 0.693232, acc.: 50.00%] [G loss: 0.691557 0.691557]
216 iter 2164 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691559 0.691559]
216 iter 2165 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691561 0.691561]
216 iter 2166 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691562 0.691562]
216 iter 2167 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691564 0.691564]
216 iter 2168 [D loss: 0.

226 iter 2265 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691720 0.691720]
226 iter 2266 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691721 0.691721]
226 iter 2267 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691723 0.691723]
226 iter 2268 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691724 0.691724]
226 iter 2269 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691726 0.691726]
227 iter 2270 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691727 0.691727]
227 iter 2271 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691729 0.691729]
227 iter 2272 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691730 0.691730]
227 iter 2273 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691732 0.691732]
227 iter 2274 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691733 0.691733]
227 iter 2275 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691735 0.691735]
227 iter 2276 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691736 0.691736]
227 iter 2277 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691738 0.691738]
227 iter 2278 [D loss: 0.

237 iter 2375 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691877 0.691877]
237 iter 2376 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691878 0.691878]
237 iter 2377 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691880 0.691880]
237 iter 2378 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691881 0.691881]
237 iter 2379 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691882 0.691882]
238 iter 2380 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691884 0.691884]
238 iter 2381 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691885 0.691885]
238 iter 2382 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691886 0.691886]
238 iter 2383 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691888 0.691888]
238 iter 2384 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691889 0.691889]
238 iter 2385 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691890 0.691890]
238 iter 2386 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691892 0.691892]
238 iter 2387 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.691893 0.691893]
238 iter 2388 [D loss: 0.

248 iter 2485 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692018 0.692018]
248 iter 2486 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692019 0.692019]
248 iter 2487 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692020 0.692020]
248 iter 2488 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692022 0.692022]
248 iter 2489 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692023 0.692023]
249 iter 2490 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692024 0.692024]
249 iter 2491 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692025 0.692025]
249 iter 2492 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692026 0.692026]
249 iter 2493 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692028 0.692028]
249 iter 2494 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692029 0.692029]
249 iter 2495 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692030 0.692030]
249 iter 2496 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692031 0.692031]
249 iter 2497 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692032 0.692032]
249 iter 2498 [D loss: 0.

259 iter 2594 [D loss: 0.693231, acc.: 50.00%] [G loss: 0.692143 0.692143]
